<a href="https://colab.research.google.com/github/cepdnaclk/e17-4yp-using-machine-learning-in-high-stake-settings/blob/main/code/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import joblib
import sys

sys.modules['sklearn.externals.joblib'] = joblib

In [ ]:
import os
import h5py
import json
import keras
import random 
import imageio
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras import backend as K
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve


## Use dataset from shared drive **XAI** I have given access to you. 

## Then we don't want to change code when loading resources.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/XAI/Dataset/DsDnsPrScTch.csv') 
data.shape
print(data.dtypes)

<ipython-input-5-a025132742c0>:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/Shareddrives/XAI/Dataset/DsDnsPrScTch.csv')


Unnamed: 0                               int64
Project ID                              object
Donation ID                             object
Donor ID                                object
Donation Included Optional Donation     object
Donation Amount                        float64
Donor Cart Sequence                      int64
Donation Received Date                  object
Donor City                              object
Donor State                             object
Donor Is Teacher                        object
Donor Zip                               object
School ID                               object
Teacher ID                              object
Teacher Project Posted Sequence          int64
Project Type                            object
Project Title                           object
Project Essay                           object
Project Short Description               object
Project Need Statement                  object
Project Subject Category Tree           object
Project Subje

In [ ]:
# Check for duplicates
data[data.duplicated()]

,Unnamed: 0,Project ID,Donation ID,Donor ID,Donation Included Optional Donation,Donation Amount,Donor Cart Sequence,Donation Received Date,Donor City,Donor State,...,School Name,School Metro Type,School Percentage Free Lunch,School State,School Zip,School City,School County,School District,Teacher Prefix,Teacher First Project Posted Date


In [ ]:
d1 = pd.read_csv('/content/drive/Shareddrives/XAI/Dataset/Donations.csv')
print(d1.columns)
print(d1.isnull().values.any())

Index(['Project ID', 'Donation ID', 'Donor ID',
       'Donation Included Optional Donation', 'Donation Amount',
       'Donor Cart Sequence', 'Donation Received Date'],
      dtype='object')
False


In [ ]:
d2 = pd.read_csv('/content/drive/Shareddrives/XAI/Dataset/Donors.csv')
print(d2.columns)
print(d2.isnull().values.any())

<ipython-input-6-434cf17defdf>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  d2 = pd.read_csv('/content/drive/Shareddrives/XAI/Dataset/Donors.csv')


Index(['Donor ID', 'Donor City', 'Donor State', 'Donor Is Teacher',
       'Donor Zip'],
      dtype='object')
True


In [ ]:
d3 = pd.read_csv('/content/drive/Shareddrives/XAI/Dataset/Projects.csv')
print(d3.columns)
print(d3.isnull().values.any())
df3 = d3.groupby('Project ID')['Project ID'].agg(count='count')
df3 = df3.loc[df3['count'] > 1].reset_index()
df3

Index(['Project ID', 'School ID', 'Teacher ID',
       'Teacher Project Posted Sequence', 'Project Type', 'Project Title',
       'Project Essay', 'Project Short Description', 'Project Need Statement',
       'Project Subject Category Tree', 'Project Subject Subcategory Tree',
       'Project Grade Level Category', 'Project Resource Category',
       'Project Cost', 'Project Posted Date', 'Project Expiration Date',
       'Project Current Status', 'Project Fully Funded Date'],
      dtype='object')
True


,Project ID,count
0,99c07777fdcf63d3a0fdb4a0deb4b012,2
1,c940d0e78b7559573aca536db90c0646,2


In [ ]:
d3.groupby('Project Current Status')['Project Current Status'].agg(count='count')

,count
Project Current Status,
Expired,241402
Fully Funded,826764
Live,41851


In [ ]:
d3.head()

,Project ID,School ID,Teacher ID,Teacher Project Posted Sequence,Project Type,Project Title,Project Essay,Project Short Description,Project Need Statement,Project Subject Category Tree,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,Project Cost,Project Posted Date,Project Expiration Date,Project Current Status,Project Fully Funded Date
0,7685f0265a19d7b52a470ee4bac883ba,e180c7424cb9c68cb49f141b092a988f,4ee5200e89d9e2998ec8baad8a3c5968,25,Teacher-Led,Stand Up to Bullying: Together We Can!,Did you know that 1-7 students in grades K-12 ...,Did you know that 1-7 students in grades K-12 ...,"My students need 25 copies of ""Bullying in Sch...",Applied Learning,"Character Education, Early Development",Grades PreK-2,Technology,361.80,2013-01-01,2013-05-30,Fully Funded,2013-01-11
1,f9f4af7099061fb4bf44642a03e5c331,08b20f1e2125103ed7aa17e8d76c71d4,cca2d1d277fb4adb50147b49cdc3b156,3,Teacher-Led,Learning in Color!,"Help us have a fun, interactive listening cent...","Help us have a fun, interactive listening cent...","My students need a listening center, read alon...","Applied Learning, Literacy & Language","Early Development, Literacy",Grades PreK-2,Technology,512.85,2013-01-01,2013-05-31,Expired,NaN
2,afd99a01739ad5557b51b1ba0174e832,1287f5128b1f36bf8434e5705a7cc04d,6c5bd0d4f20547a001628aefd71de89e,1,Teacher-Led,Help Second Grade ESL Students Develop Languag...,Visiting or moving to a new place can be very ...,Visiting or moving to a new place can be very ...,My students need beginning vocabulary audio ca...,Literacy & Language,ESL,Grades PreK-2,Supplies,435.92,2013-01-01,2013-05-30,Fully Funded,2013-05-22
3,c614a38bb1a5e68e2ae6ad9d94bb2492,900fec9cd7a3188acbc90586a09584ef,8ed6f8181d092a8f4c008b18d18e54ad,40,Teacher-Led,Help Bilingual Students Strengthen Reading Com...,Students at our school are still working hard ...,Students at our school are still working hard ...,My students need one copy of each book in The ...,Literacy & Language,"ESL, Literacy",Grades 3-5,Books,161.26,2013-01-01,2013-05-31,Fully Funded,2013-02-06
4,ec82a697fab916c0db0cdad746338df9,3b200e7fe3e6dde3c169c02e5fb5ae86,893173d62775f8be7c30bf4220ad0c33,2,Teacher-Led,Help Us Make Each Minute Count!,"""Idle hands"" were something that Issac Watts s...","""Idle hands"" were something that Issac Watts s...","My students need items such as Velcro, two pou...",Special Needs,Special Needs,Grades 3-5,Supplies,264.19,2013-01-01,2013-05-30,Fully Funded,2013-01-01


In [ ]:
d4 = pd.read_csv('/content/drive/Shareddrives/XAI/Dataset/Resources.csv')
print(d4.columns)
print(d4.isnull().values.any())

In [ ]:
d4[pd.isnull(d4["Resource Item Name"])]

,Project ID,Resource Item Name,Resource Quantity,Resource Unit Price,Resource Vendor Name
15798,008c73956e40845a81acec28b578dd0a,NaN,NaN,NaN,NaN
23914,00d18fccd06542c715313a51ec496895,NaN,1.0,9.95,Blick Art Materials
30180,0106359fa2c04bb528f8aaabd27d736c,NaN,1.0,9.95,Blick Art Materials
38271,014b18458841ae8732a36c41a5b57050,NaN,NaN,NaN,NaN
54639,01dfe3f303bdaaf55dfde85606160d1c,NaN,10.0,1.27,Blick Art Materials
...,...,...,...,...,...
7165673,fe6924afcf0fa643209719601226a5d9,NaN,2.0,24.53,Blick Art Materials
7168478,fe83619e112945503217fee4c2c91cde,NaN,NaN,NaN,NaN
7179167,fee376e145682a3453145f1d5617d5f7,NaN,1.0,9.95,Blick Art Materials
7182005,fefe22769f76db3dbb34a4f4a7f4ecc8,NaN,5.0,11.99,Amazon Business


In [ ]:
d5 = pd.read_csv('/content/drive/Shareddrives/XAI/Dataset/Schools.csv')
print(d5.columns)
print(d5.isnull().values.any())

Index(['School ID', 'School Name', 'School Metro Type',
       'School Percentage Free Lunch', 'School State', 'School Zip',
       'School City', 'School County', 'School District'],
      dtype='object')
True


In [ ]:
d5[pd.isnull(d5["School City"])]

,School ID,School Name,School Metro Type,School Percentage Free Lunch,School State,School Zip,School City,School County,School District
287,0105b4eff46d80be26b08489ba173ac9,Briya Public Charter School - Adams Morgan,urban,98.0,District of Columbia,20009,NaN,District Of Columbia,Dc Public Schools
618,021ba96a0f6cbbe0a992ea8f8f6ebf24,Choice Academy,urban,NaN,District of Columbia,20002,NaN,District Of Columbia,Dc Public Schools
760,028b4f7a4087d0b799f496d88460b59a,Murch Elementary School,urban,7.0,District of Columbia,20008,NaN,District Of Columbia,Dc Public Schools
981,03459ee7b2faf5ca25af3b36b295b4a2,M C Terrell Elementary School,urban,95.0,District of Columbia,20032,NaN,District Of Columbia,Dc Public Schools
1478,04f758bb12efab4d0257aa232ff10335,KIPP DC-Northeast Academy,urban,77.0,District of Columbia,20002,NaN,District Of Columbia,[State Run]
...,...,...,...,...,...,...,...,...,...
72018,fc72ed1889debe4f72adb7af33c62c26,Sustainable Futures Public Charter School,unknown,NaN,District of Columbia,20009,NaN,District Of Columbia,Dc Public Schools
72535,fe5554ed497a0c43e6094fbe98318b8f,Friendship Public Charter School- Woodridge,urban,49.0,District of Columbia,20018,NaN,District Of Columbia,Dc Public Schools
72776,ff32fa746e6ce3c8f4a02eb9ee208ade,Capital City Public Charter School-High School,urban,NaN,District of Columbia,20011,NaN,District Of Columbia,[State Run]
72846,ff7c502e980cc815e5b1bd06f13bd570,Center City Public Charter School (Shaw),urban,NaN,District of Columbia,20001,NaN,District Of Columbia,[State Run]


In [ ]:
d6 = pd.read_csv('/content/drive/Shareddrives/XAI/Dataset/Stat_Data.csv')
print(d6.columns)
print(d6.isnull().values.any())

<ipython-input-27-e1c60bfcd715>:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  d6 = pd.read_csv('/content/drive/Shareddrives/XAI/Dataset/Stat_Data.csv')


Index(['Unnamed: 0', 'Donation ID', 'Project ID', 'Donor ID', 'School ID',
       'Teacher ID', 'Donation Amount', 'Project Cost', 'Project Type',
       'Project Subject Category Tree', 'Project Subject Subcategory Tree',
       'Project Grade Level Category', 'Project Resource Category',
       'Teacher Project Posted Sequence', 'Donor Zip', 'Donor City',
       'Donor State', 'Donor Cart Sequence', 'Donor Is Teacher',
       'Project Current Status', 'School Metro Type', 'School Zip',
       'School State', 'Teacher Prefix', 'School Percentage Free Lunch',
       'Donation Included Optional Donation'],
      dtype='object')
True


In [ ]:
d7 = pd.read_csv('/content/drive/Shareddrives/XAI/Dataset/Teachers.csv')
print(d7.columns)
print(d7.isnull().values.any())

Index(['Teacher ID', 'Teacher Prefix', 'Teacher First Project Posted Date'], dtype='object')
True


In [ ]:
data.columns
data.head()

,Unnamed: 0,Project ID,Donation ID,Donor ID,Donation Included Optional Donation,Donation Amount,Donor Cart Sequence,Donation Received Date,Donor City,Donor State,...,School Name,School Metro Type,School Percentage Free Lunch,School State,School Zip,School City,School County,School District,Teacher Prefix,Teacher First Project Posted Date
0,0,000009891526c0ade7180f8423792063,688729120858666221208529ee3fc18e,1f4b5b6e68445c6c4a0509b3aca93f38,No,178.37,11,2016-08-23 13:15:57,West Jordan,Utah,...,Oquirrh Hills Middle School,suburban,23.0,Utah,84065,Riverton,Salt Lake,Jordan School District,Mr.,2015-09-19
1,1,000009891526c0ade7180f8423792063,dcf1071da3aa3561f91ac689d1f73dee,4aaab6d244bf3599682239ed5591af8a,Yes,25.00,2,2016-06-06 20:05:23,Riverton,Utah,...,Oquirrh Hills Middle School,suburban,23.0,Utah,84065,Riverton,Salt Lake,Jordan School District,Mr.,2015-09-19
2,2,000009891526c0ade7180f8423792063,18a234b9d1e538c431761d521ea7799d,0b0765dc9c759adc48a07688ba25e94e,Yes,20.00,3,2016-06-06 14:08:46,Riverton,Utah,...,Oquirrh Hills Middle School,suburban,23.0,Utah,84065,Riverton,Salt Lake,Jordan School District,Mr.,2015-09-19
3,3,000009891526c0ade7180f8423792063,38d2744bf9138b0b57ed581c76c0e2da,377944ad61f72d800b25ec1862aec363,Yes,25.00,1,2016-05-15 10:23:04,Phoenix,Arizona,...,Oquirrh Hills Middle School,suburban,23.0,Utah,84065,Riverton,Salt Lake,Jordan School District,Mr.,2015-09-19
4,4,000009891526c0ade7180f8423792063,5a032791e31167a70206bfb86fb60035,6d5b22d39e68c656071a842732c63a0c,Yes,25.00,2,2016-05-17 01:23:38,Salt Lake City,Utah,...,Oquirrh Hills Middle School,suburban,23.0,Utah,84065,Riverton,Salt Lake,Jordan School District,Mr.,2015-09-19


In [ ]:
data.shape
data.isnull().sum()

Unnamed: 0                                  0
Project ID                                  0
Donation ID                                 0
Donor ID                                    0
Donation Included Optional Donation         0
Donation Amount                             0
Donor Cart Sequence                         0
Donation Received Date                      0
Donor City                             346083
Donor State                                 0
Donor Is Teacher                            0
Donor Zip                              267808
School ID                                   0
Teacher ID                                  0
Teacher Project Posted Sequence             0
Project Type                                0
Project Title                              34
Project Essay                              10
Project Short Description                  19
Project Need Statement                      2
Project Subject Category Tree             111
Project Subject Subcategory Tree  

In [ ]:
raw_dataset = [list(d1.columns), list(d2.columns), list(d3.columns), list(d4.columns), list(d5.columns), list(d6.columns), list(d7.columns)]

In [ ]:
unique_cols = []
for d in raw_dataset:
  unique_cols += d
unique_cols = list(set(unique_cols))
print(unique_cols)
print(len(unique_cols))

Imputing missing values:
When missing values is from categorical columns (string or numerical) then the missing values can be 
replaced with the most frequent category. 
If the number of missing values is very large then it can be replaced with a new category.

- Donor City (346083)- New category
- Donor Zip (267808) - New category
- Project Title (34) - Drop column 
- project Essay (10) - Drop column
- Project Short Description (19) - Drop column
- Project Need Statement (2) - Drop column

- Project Subject Category Tree (111) - 
- Project Subject Subcategory Tree (111)
For these two columns, we can seperate the comma-seperated categories and create new binary columns

- Project Resource Category (134) - Can use the most frequent item
- Project Expiration Date (3) - https://www.analyticsvidhya.com/blog/2023/02/impute-missing-dates-not-data-in-python/

- School Percentage Free Lunch (21894) - Mean/Median
- School City (30146) - New category
- School County (37) - Most frequent
- Teacher Prefix (139) - Most frequent


New possible features (granularity - Project ID):

- Number of donations made during the selected time window
- Number of donors in the selected time window
- Most frequent donor city/state in the selected time window
- Donor is teacher yes/no count in the selected time window

In [ ]:
# Data preprocessing

# Set data types for datetime
data["Teacher First Project Posted Date"]=pd.to_datetime(data["Teacher First Project Posted Date"])
data["Project Fully Funded Date"]=pd.to_datetime(data["Project Fully Funded Date"])
data["Project Expiration Date"]=pd.to_datetime(data["Project Expiration Date"])
data["Project Posted Date"]=pd.to_datetime(data["Project Posted Date"])
data["Donation Received Date"]=pd.to_datetime(data["Donation Received Date"])
data["Number of dates since posted"]=data.loc[:,"Donation Received Date"] - data.loc[:,"Project Posted Date"]

In [ ]:
data["Project Posted Date"].max()
#data["Project Posted Date"].min()

Timestamp('2018-05-01 00:00:00')

In [ ]:
# Imputing missing data
data["Donor City"] = data["Donor City"].fillna("Unknown")
data["Donor Zip"] = data["Donor Zip"].fillna("Unknown")

data["School Percentage Free Lunch"] = data["School Percentage Free Lunch"].replace(np.NaN, data["School Percentage Free Lunch"].median())

data["School City"] = data["School City"].fillna("Unknown")
data["School County"] = data["School County"].fillna(data["School County"].mode()[0])
data["Teacher Prefix"] = data["Teacher Prefix"].fillna(data["Teacher Prefix"].mode()[0])


In [ ]:
data.shape

(4608080, 40)

In [ ]:
# Function for one hot encoding in a specific time window
def one_hot_encoding_columns(df, cat_cols_with_nan):

  # For the categorical variables with one/more value(s) for each row
  for col in cat_cols_with_nan:
    modified_df = pd.concat(
        [df, 
         df[col].str.get_dummies(sep=', ').add_prefix(col + " ")], axis=1
         )

  modified_df = modified_df.drop(cat_cols_with_nan, axis=1)  

  return modified_df

In [ ]:
# Function for scaling after seperating into train/test
def standardize_data(x_train, x_test, cols_list):

  # Create scaler
  ss = StandardScaler()
  features_train = x_train[cols_list]
  features_test = x_test[cols_list]

  # Fit scaler only for training data
  scaler = ss.fit(features_train)

  # Transform training data
  features_t = scaler.transform(features_train)
  x_train[cols_list] = features_t

  # Transforming test data
  for_test = scaler.transform(features_test)
  x_test[cols_list] = for_test

  return x_train, x_test

In [ ]:
# Function for feature selection
def run_sfs(x, y):

  # Define classifier
  knn = KNeighborsClassifier(n_neighbors=4)
  sfs1 = sfs(knn, 
             k_features="best", 
             forward=True, 
             floating=False, 
             verbose=2,
             scoring='accuracy',
             cv=5)
  sfs1 = sfs1.fit(x, y)
  feature_names = list(sfs1.k_feature_names_)
  print(feature_names)
  print(sfs1.k_score_)

  return feature_names

In [ ]:
def run_model_with_gridsearch(classifier, parameters, x_train, y_train, x_test, y_test, scoring):
  # Initiate model
  model = GridSearchCV(classifier, parameters, cv=5, scoring=scoring)
  # Fit the model on training data
  model.fit(x_train, y_train)

  print("Best parameters: ", model.best_params_)
  print("Scoring: ", model.best_score_)

  # Predict for test data
  y_hat = model.predict(x_test)


  return y_hat, model


In [ ]:
# Test pipeline function
def run_pipeline(data, model, parameters, time_period_in_days, fund_ratio_threshold, categorical_variables, variables_to_scale):

  # Initiate lists to store data
  t_current_list = []

  # Create new features
  data["Posted Date to Donation Date"] = data.loc[:,"Donation Received Date"] - data.loc[:,"Project Posted Date"]
  data["Posted Date to Donation Date"] = data["Posted Date to Donation Date"] / np.timedelta64(1, 'D')

  # Initiate variables
  max_t = pd.Timestamp("2013-05-01 00:00:00") #"2018-05-01 00:00:00"
  min_t = pd.Timestamp("2013-01-01 00:00:00")
  time_period = timedelta(days=time_period_in_days)  
  time_posted_window = timedelta(days=120)
  t_current = min_t - time_period + time_posted_window # min-t + three months
  # One hot encoding
  data = one_hot_encoding_columns(data, categorical_variables)

  # Start cohort
  while(t_current < max_t - time_period):

    # Adjust the current time
    t_current = t_current + time_period
    print("Current date: ", t_current)
    t_current_list += [t_current]
    t_start = t_current - time_posted_window # t_start = min_t
    t_end = t_current - time_period

    # Filter rows for the relevant time period
    data_window = data[data["Project Posted Date"] < pd.to_datetime(t_current)]
    data_window = data_window[data_window["Project Posted Date"] > pd.to_datetime(t_start)]
    data_window = data_window[data_window["Posted Date to Donation Date"] < time_period_in_days]

    # Create new features
    data_window["Donation to Cost"] = data_window["Donation Amount"] / data_window["Project Cost"]

    # Create new features by aggregating
    data_window["Total Donations"] = data_window.groupby("Project ID")["Donation Amount"].transform("sum")
    data_window["Number of Donors"] = data_window.groupby("Project ID")["Donor ID"].transform("count")
    data_window["Fund Ratio"] = data_window.groupby("Project ID")["Donation to Cost"].transform("sum")
    data_window["Donor State Most Frequent"] = data_window.groupby("Project ID")["Donor State"].transform(lambda x: x.value_counts().idxmax())
    data_window["Teacher is Donor Count"] = (data_window["Donor Is Teacher"] == 'Yes').groupby("Project ID")["Donor Is Teacher"].transform("count")
    data_window["Teacher is Donor Ratio"] = data_window["Teacher is Donor Count"] / data_window["Number of Donors"]

    # Create label
    data_window["Label"] = data_window.apply(lambda x : 1  if x["Fund Ratio"] < fund_ratio_threshold  else 0, axis=1)

    # Filter for columns and remove duplicate rows
    cols_to_select = ["Project Id", "Total Donations", "Number of Donors", "Fund Ratio", "Project Type", "Project Subject Category Tree", 
                  "Project Subject Subcategory Tree", "Project Grade Level Category", "Project Resource Category", "Donor State Most Frequent", 
                  "Donor City Most Frequent", "School Metro Type", "School Percentage Free Lunch", "School State", "School County", 
                  "Teacher Prefix", "Teacher is Donor Ratio", "Label"]
    data_window_final = data_window[cols_to_select].drop_duplicates()

    # One hot encoding
    data_window_final = one_hot_encoding_columns(data_window_final, categorical_variables)

    # Create training set
    train_set = data_window_final[data_window_final["Project Posted Date"] < pd.to_datetime(t_end)]
    model_vars = ["Project Id", "Total Donations", "Number of Donors", "Fund Ratio", "Project Type", "Project Subject Category Tree", 
                  "Project Subject Subcategory Tree", "Project Grade Level Category", "Project Resource Category", "Donor State Most Frequent", 
                  "Donor City Most Frequent", "School Metro Type", "School Percentage Free Lunch", "School State", "School County", 
                  "Teacher Prefix", "Teacher is Donor Ratio"]
    x_train = train_set.loc[:, model_vars]
    y_train = train_set.loc[:, ["Label"]]

    # Create testing set
    test_set = data_window_final[data_window_final["Project Posted Date"] >= pd.to_datetime(t_end)]
    x_test = test_set.loc[:, model_vars]
    y_test = test_set.loc[:, ["Label"]]

    # Scaling
    x_train, x_test = standardize_data(x_train, x_test, variables_to_scale)

    # Run SFS

    # Run model
    y_hat, model_post_run = run_model_with_gridsearch(model, parameters, x_train, y_train, x_test, y_test, "accuracy")

    # Evaluate
    cm = confusion_matrix(y_test, y_hat)
    sns.heatmap(cm, square=True, annot=True, cbar=False)
    plt.xlabel('Predicted Value')
    plt.ylabel('Actual Value')
    plt.show()

  return